In [1]:
import boto3
import os
import sagemaker
import sagemaker.session
import datetime
import json
import pathlib
import yaml
from sagemaker.xgboost import XGBoostProcessor
from sagemaker.estimator import Estimator
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import TrainingStep
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.parameters import (
    ParameterString)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
role = 'arn:aws:iam::266735847038:role/sagemakerrole'
model_path = 's3://mbucket100/model-path/'


# In[3]:


steps=[]

xgb = XGBoostProcessor(
            framework_version='1.7-1',
            py_version="py3",
            role=role,
            instance_type='ml.t3.medium',
            instance_count=1,
            sagemaker_session=PipelineSession(),
            base_job_name='Manasjob'
        )
        
xgb_description = xgb.run(
            code = 'train.py',
            source_dir = 'train',
    
            outputs=[
                ProcessingOutput(
                    output_name='train_output_s3',
                    source='/opt/ml/processing/output/train/',
                    destination= f'{model_path}'
                )
            ],
        )
        
train_step = ProcessingStep(name = 'train_step',step_args = xgb_description)

steps.append(train_step) #to_maintain the order

# model_approval_status = ParameterString(name="ModelApprovalStatus",default_value='ModelApprovalStatus')

print("=====================>pipeline_name : Manas Training")
print(steps)
pipeline = Pipeline(
                        name = 'ManasTraining',
                        steps=steps,
                        # parameters = [
                        #     model_approval_status
                        # ]
                    )
print("=====================>Training Pipeline created : Definition")
print(json.dumps(pipeline.definition()))
print("=====================>Role upsert")
pipeline.upsert(role_arn=role)
body = {
    "message": "Sagemaker Training Pipeline Creation Successfully!",
    "input": "Manas-Training"
}
pipeline.start()
print(f'Upserted {body}')

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.t3.medium.
/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


=====================>pipeline_name : Manas Training
=====================>Training Pipeline created : Definition


INFO:sagemaker.processing:Uploaded train to s3://sagemaker-ap-south-1-266735847038/ManasTraining/code/a02f77ab6b5dc57e88eac97b68bf0061/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-south-1-266735847038/ManasTraining/code/44b98885aa3a483f9b59278694840a27/runproc.sh
INFO:sagemaker.processing:Uploaded train to s3://sagemaker-ap-south-1-266735847038/ManasTraining/code/a02f77ab6b5dc57e88eac97b68bf0061/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-south-1-266735847038/ManasTraining/code/44b98885aa3a483f9b59278694840a27/runproc.sh


"{\"Version\": \"2020-12-01\", \"Metadata\": {}, \"Parameters\": [], \"PipelineExperimentConfig\": {\"ExperimentName\": {\"Get\": \"Execution.PipelineName\"}, \"TrialName\": {\"Get\": \"Execution.PipelineExecutionId\"}}, \"Steps\": [{\"Name\": \"train_step\", \"Type\": \"Processing\", \"Arguments\": {\"ProcessingResources\": {\"ClusterConfig\": {\"InstanceType\": \"ml.t3.medium\", \"InstanceCount\": 1, \"VolumeSizeInGB\": 30}}, \"AppSpecification\": {\"ImageUri\": \"720646828776.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-xgboost:1.7-1\", \"ContainerEntrypoint\": [\"/bin/bash\", \"/opt/ml/processing/input/entrypoint/runproc.sh\"]}, \"RoleArn\": \"arn:aws:iam::266735847038:role/sagemakerrole\", \"ProcessingInputs\": [{\"InputName\": \"code\", \"AppManaged\": false, \"S3Input\": {\"S3Uri\": \"s3://sagemaker-ap-south-1-266735847038/ManasTraining/code/a02f77ab6b5dc57e88eac97b68bf0061/sourcedir.tar.gz\", \"LocalPath\": \"/opt/ml/processing/input/code/\", \"S3DataType\": \"S3Prefix\", \"S3Inp

INFO:sagemaker.processing:Uploaded train to s3://sagemaker-ap-south-1-266735847038/ManasTraining/code/a02f77ab6b5dc57e88eac97b68bf0061/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-south-1-266735847038/ManasTraining/code/44b98885aa3a483f9b59278694840a27/runproc.sh


Upserted {'message': 'Sagemaker Training Pipeline Creation Successfully!', 'input': 'Manas-Training'}


In [15]:
import boto3
from sagemaker.workflow.pipeline import Pipeline

# Define your pipeline name
pipeline_name = "ManasTraining"

# Initialize a SageMaker session
sagemaker_session = sagemaker.Session()
client = boto3.client('sagemaker')

# Describe the pipeline
pipeline_description = client.describe_pipeline(
    PipelineName=pipeline_name
)

# Retrieve the pipeline's latest execution details
pipeline_executions = client.list_pipeline_executions(
    PipelineName=pipeline_name
)

# Get the status of the most recent execution
if pipeline_executions['PipelineExecutionSummaries']:
    latest_execution = pipeline_executions['PipelineExecutionSummaries'][0]
    status = latest_execution['PipelineExecutionStatus']
    start_time = latest_execution['StartTime']
    end_time = latest_execution.get('EndTime', 'Still running')

    print(f"Pipeline: {pipeline_name}")
    print(f"Status: {status}")
    print(f"Start Time: {start_time}")
    print(f"End Time: {end_time}")
else:
    print(f"No executions found for the pipeline: {pipeline_name}")

Pipeline: ManasTraining
Status: Failed
Start Time: 2024-08-28 08:08:05.971000+00:00
End Time: Still running


In [7]:
pipeline_description

{'PipelineArn': 'arn:aws:sagemaker:ap-south-1:266735847038:pipeline/ManasTraining',
 'PipelineName': 'ManasTraining',
 'PipelineDisplayName': 'ManasTraining',
 'PipelineDefinition': '{"Version": "2020-12-01", "Metadata": {}, "Parameters": [], "PipelineExperimentConfig": {"ExperimentName": {"Get": "Execution.PipelineName"}, "TrialName": {"Get": "Execution.PipelineExecutionId"}}, "Steps": [{"Name": "train_step", "Type": "Processing", "Arguments": {"ProcessingResources": {"ClusterConfig": {"InstanceType": "ml.t3.medium", "InstanceCount": 1, "VolumeSizeInGB": 30}}, "AppSpecification": {"ImageUri": "720646828776.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-xgboost:1.7-1", "ContainerEntrypoint": ["/bin/bash", "/opt/ml/processing/input/entrypoint/runproc.sh"]}, "RoleArn": "arn:aws:iam::266735847038:role/sagemakerrole", "ProcessingInputs": [{"InputName": "code", "AppManaged": false, "S3Input": {"S3Uri": "s3://sagemaker-ap-south-1-266735847038/ManasTraining/code/a02f77ab6b5dc57e88eac97b68bf0061/s

In [9]:
import boto3

# Define your pipeline name and initialize the SageMaker client
pipeline_name = "ManasTraining"
client = boto3.client('sagemaker')

# Get the list of executions
pipeline_executions = client.list_pipeline_executions(
    PipelineName=pipeline_name
)

# Find the most recent execution
if pipeline_executions['PipelineExecutionSummaries']:
    latest_execution = pipeline_executions['PipelineExecutionSummaries'][0]
    pipeline_execution_arn= latest_execution['PipelineExecutionArn']

    # Describe the pipeline execution
    execution_details = client.describe_pipeline_execution(
        PipelineExecutionArn=pipeline_execution_arn
    )
    
    print(f"Pipeline Execution Status: {execution_details['PipelineExecutionStatus']}")
    
    # Check for failure reasons
    if execution_details['PipelineExecutionStatus'] == 'Failed':
        print("Failure Reason:", execution_details.get('FailureReason', 'No specific reason provided'))

        # Get details of each step in the pipeline
        steps = execution_details['PipelineExecutionDescription']
        for step in steps:
            step_name = step['StepName']
            step_status = step['StepStatus']
            if step_status == 'Failed':
                print(f"Step: {step_name} failed.")
                print(f"Failure Reason: {step.get('FailureReason', 'No specific reason provided')}")
                print(f"Error Details: {step.get('FailureDetails', 'No additional error details')}")
    else:
        print("No failure details available. The execution status is not 'Failed'.")
else:
    print(f"No executions found for the pipeline: {pipeline_name}")

Pipeline Execution Status: Failed
Failure Reason: Step failure: One or multiple steps failed.


KeyError: 'PipelineExecutionDescription'

In [10]:
execution_details

{'PipelineArn': 'arn:aws:sagemaker:ap-south-1:266735847038:pipeline/ManasTraining',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-south-1:266735847038:pipeline/ManasTraining/execution/7splq4srxncr',
 'PipelineExecutionDisplayName': 'execution-1724831222929',
 'PipelineExecutionStatus': 'Failed',
 'PipelineExperimentConfig': {'ExperimentName': 'manastraining',
  'TrialName': '7splq4srxncr'},
 'FailureReason': 'Step failure: One or multiple steps failed.',
 'CreationTime': datetime.datetime(2024, 8, 28, 7, 47, 2, 839000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 8, 28, 7, 47, 4, 734000, tzinfo=tzlocal()),
 'CreatedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::266735847038:assumed-role/sagemakerrole/SageMaker',
   'PrincipalId': 'AROAT4GVSAZ7FI6TUYBPF:SageMaker'}},
 'LastModifiedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::266735847038:assumed-role/sagemakerrole/SageMaker',
   'PrincipalId': 'AROAT4GVSAZ7FI6TUYBPF:SageMaker'}},
 'ResponseMetadata': {'RequestId': '95217be

In [ ]:
53